In [1]:
import pandas as pd
import numpy as np

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

from sentence_transformers import util


import fasttext
import fasttext.util

from sklearn.metrics import accuracy_score

/Users/kapeleshh/.conda/envs/STS/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  after removing the cwd from sys.path.


In [2]:
SEED = 13
np.random.seed(SEED)

### Load Data

In [3]:
filepath = "/Users/kapeleshh/PycharmProjects/STS/"
df = pd.read_pickle(filepath + "data/clinic_sudachi")
xls =  pd.ExcelFile(filepath + "data/dataset_JA_Clinical_STS.xls")
dfs = xls.parse(0)
dfs.columns = ['s1', 's2', 'score']
df['score'] = dfs['score']

In [4]:
pretrained_path = filepath + "models/cc.ja.300.bin"
fasttext_model = fasttext.load_model(pretrained_path)

In [5]:
df['cs_f'] = ''
for i in range(df.shape[0]):
    embeddings1 = np.mean([fasttext_model[x] for x in df['s1'].iloc[i]], axis=0)
    embeddings2 = np.mean([fasttext_model[x] for x in df['s2'].iloc[i]], axis=0)
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    df['cs_f'].iloc[i] = cosine_scores.item()



/Users/kapeleshh/.conda/envs/STS/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
df['cs_f'] = df['cs_f'].apply(lambda x: int(round(x*5, 0)))

In [7]:
print("Accuracy of Train cs_f:", accuracy_score(dfs['score'], df['cs_f']))




Accuracy of Train cs_f: 0.20741346415917145
